<a href="https://colab.research.google.com/github/FabianoNegreiros/aluraFN/blob/main/AgenteIAbanco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
%pip -q install google-genai
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.

In [3]:
import os
from google.colab import userdata
from google import genai
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types
from datetime import date
import textwrap
from IPython.display import display, Markdown
import warnings

warnings.filterwarnings("ignore")


In [4]:
# Configura a API Key do Google Gemini
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

# Configura o cliente da SDK do Gemini
client = genai.Client()

MODEL_ID = "gemini-2.0-flash"


In [5]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [6]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [7]:
######################################################
# --- Agente 1: Boas-Vindas e Identificação --- #
######################################################
def agente_boas_vindas():
    boas_vindas_agente = Agent(
        name="agente_boas_vindas",
        model=MODEL_ID,
        description="Agente que cumprimenta o cliente e coleta informações básicas.",
        instruction="""
            Você é um agente virtual de suporte bancário. Sua primeira tarefa é dar boas-vindas ao cliente
            e perguntar como você pode ajudar hoje. Seja cordial e profissional.
            Tente identificar brevemente o motivo do contato do cliente.
            """
    )
    return boas_vindas_agente


In [8]:
###########################################
# --- Agente 2: Triagem de Dúvidas --- #
###########################################
def agente_triagem():
    triagem_agente = Agent(
        name="agente_triagem",
        model=MODEL_ID,
        description="Agente que analisa a pergunta do cliente e a categoriza.",
        instruction="""
            Você é um agente virtual de suporte bancário. Analise a pergunta do cliente para
            identificar a categoria principal do problema ou dúvida. As categorias possíveis são:

            -   Cartões: Inclui problemas com cartão de crédito, débito, bloqueio, desbloqueio, etc.
            -   Empréstimos: Inclui dúvidas sobre tipos de empréstimo, taxas, simulações, etc.
            -   Financiamentos: Inclui questões sobre financiamento de imóveis, veículos, etc.
            -   Investimentos: Inclui informações sobre opções de investimento, rendimentos, etc.
            -   Contas: Inclui problemas com abertura de conta, saldo, extrato, etc.
            -   Pagamentos: Inclui dúvidas sobre boletos, transferências, etc.
            -   Outros: Para assuntos que não se encaixam nas categorias acima.

            Responda com uma frase curta indicando a categoria principal e, em seguida,
            pergunte como você pode ajudar especificamente.
            """
    )
    return triagem_agente

In [9]:
##################################################
# --- Agente 3: Respostas e Informações --- #
##################################################

def agente_respostas():
    respostas_agente = Agent(
        name="agente_respostas",
        model=MODEL_ID,
        description="Agente que fornece respostas e informações detalhadas ao cliente.",
        instruction="""
            Você é um agente virtual de suporte bancário. Forneça informações detalhadas e
            úteis para responder à pergunta do cliente. Use um tom claro e objetivo.
            Se a pergunta for sobre:

             - Cartões: Explique sobre os tipos de cartões, limites, como solicitar,
               como desbloquear, como contestar uma compra, etc.
             - Empréstimos: Detalhe os tipos de empréstimos (pessoal, consignado, etc.),
               taxas de juros, prazos de pagamento, como solicitar, etc.
             - Financiamentos: Informe sobre financiamento de imóveis e veículos,
                documentação, taxas, prazos, etc.
             - Investimentos: Explique sobre as opções de investimento (poupança, CDB,
               fundos, etc.), riscos, rentabilidade, como investir, etc.
             - Contas: Ajude com abertura de conta, saldo, extrato, tarifas, encerramento, etc.
             - Pagamentos: Informe sobre pagamentos de boletos, transferências (TED, PIX),
               agendamentos, comprovantes, etc.
            - Outros: Direcione o cliente para o atendimento humano ou forneça
              informações de contato relevantes.

            Se não souber a resposta exata, diga que vai direcionar o cliente para um
            atendente humano e forneça o número de telefone da central de atendimento.
            """
    )
    return respostas_agente

In [10]:
def executar_atendimento():
    # Cria instâncias dos agentes
    agente_inicio = agente_boas_vindas()
    agente_triagem_duvidas = agente_triagem()
    agente_resposta = agente_respostas()

    print("🤖 Iniciando o Agente Virtual de Suporte Bancário 🤖")

    # Simula a primeira interação com o agente de boas-vindas
    primeira_mensagem = "Olá!"
    resposta_boas_vindas = call_agent(agente_inicio, primeira_mensagem)
    print("\n--- Agente de Boas-Vindas ---\n")
    display(to_markdown(resposta_boas_vindas))
    print("-----------------------------\n")

    # Coleta a resposta inicial do usuário
    pergunta_usuario = input("Como posso ajudar você hoje? ")

    # Loop principal de atendimento
    while True:
        # Envia a pergunta do usuário para o agente de triagem
        resposta_triagem = call_agent(agente_triagem_duvidas, pergunta_usuario)
        print("\n--- Agente de Triagem ---\n")
        display(to_markdown(resposta_triagem))
        print("------------------------\n")

        # Envia a pergunta do usuário para o agente de respostas
        resposta_final = call_agent(agente_resposta, pergunta_usuario)
        print("\n--- Agente de Respostas ---\n")
        display(to_markdown(resposta_final))
        print("---------------------------\n")

        # Pergunta se o usuário precisa de mais ajuda ou quer encerrar
        pergunta_usuario = input("Precisa de mais alguma ajuda? (Sim/Não) ")
        if pergunta_usuario.lower() in ["não", "nao"]:
            print("Obrigado pelo contato! Tenha um bom dia.")
            break
        elif pergunta_usuario.lower() != "sim":
            print("Desculpe, não entendi. Por favor, responda com 'Sim' ou 'Não'.")
            pergunta_usuario = input("Precisa de mais alguma ajuda? (Sim/Não)")
        else:
           pergunta_usuario = input("Qual a sua próxima pergunta? ")

if __name__ == "__main__":
    executar_atendimento()

🤖 Iniciando o Agente Virtual de Suporte Bancário 🤖

--- Agente de Boas-Vindas ---



> Olá! Boas-vindas ao nosso canal de atendimento. 😊 É um prazer atendê-lo(a).
> 
> Em que posso ajudar você hoje?


-----------------------------

Como posso ajudar você hoje? Gostaria de saber aonde melhor faço investimentos

--- Agente de Triagem ---



> Investimentos. Como posso te ajudar especificamente com informações sobre investimentos?
> 


------------------------


--- Agente de Respostas ---



...

---------------------------

Precisa de mais alguma ajuda? (Sim/Não) 
Desculpe, não entendi. Por favor, responda com 'Sim' ou 'Não'.
Precisa de mais alguma ajuda? (Sim/Não)nao

--- Agente de Triagem ---



> Outros: Como posso ajudar você hoje?


------------------------


--- Agente de Respostas ---



> Por favor, pode especificar sua pergunta ou o que você precisa? Assim, posso fornecer as informações detalhadas e úteis que você procura.


---------------------------

Precisa de mais alguma ajuda? (Sim/Não) Não
Obrigado pelo contato! Tenha um bom dia.
